# Desafio - Aula 3

### Nome: Jonny Silva Canatto
### RM: 332439

In [13]:
#Realizando todos os imports necessários
import pandas as pd
import numpy as np
import pandas as pd
import nltk
import spacy
import re
import os, importlib

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import floresta
from nltk.stem.wordnet import WordNetLemmatizer

from collections import Counter

nltk.download('punkt')
nlp = spacy.load('pt_core_news_sm')
nltk.download('floresta')
stops = nltk.corpus.stopwords.words('english')
ps = PorterStemmer()
wnl = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package floresta to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package floresta is already up-to-date!


## 1) Crie uma classe de tratamento de texto. A classe deverá conter:
   ## -> remoção de números
   ## -> passar o texto para caixa baixa
   ## -> remoção de caracteres especiais
   ## -> remoção de stop-words
   ## -> Stemização/lemmatização (deve ser passado como parâmetro qual abordagem utilizar)

In [16]:
# resposta 1
    
def remove_numero_caracteres_especiais(linha):
    linha = re.sub(r'[^a-zA-Z ]+','',linha)
    return linha

def caixa_baixa(linha):
    linha = linha.lower()
    return linha

def remove_stop_words(linha):
    linha = ' '.join([palavra for palavra in linha.split(' ') if palavra not in stops])
    return linha

def lemmatizacao(linha):

    linha = ' '.join([wnl.lemmatize(palavra, 'v') for palavra in linha.split(' ')])
    return linha

def stemizacao(linha):
    linha = ' '.join([ps.stem(palavra) for palavra in word_tokenize(linha)])
    return linha

def tratarDataFrame(linha,stem = False,lem = False):
    linha = caixa_baixa(linha)
    linha = remove_numero_caracteres_especiais(linha)
    linha = remove_stop_words(linha)
    if(stem):
        linha = stemizacao(linha)
    if(lem):
        linha = lemmatizacao(linha)
    return linha

## 2) Refaça o exercício de aula (movie_review) realizando o tratamento do texto antes. Houve diferença? Descreva

In [17]:
# resposta 2

#Leitura do dataFrame
data = pd.read_csv('movie_review1.csv',index_col=0)

#Limpeza dos dados
data['text'] = [tratarDataFrame(txt, True, False) for txt in data['text']]

print('Visualização dos dados:')
print(data.head())
print('------------------------')
#Montando BOW
vet = TfidfVectorizer()
bow = vet.fit_transform(data['text'])

#Definições de treino
X_train, X_test, y_train, y_test = train_test_split(bow, data['tag'], test_size=0.3, random_state=69, shuffle = True)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)

#Resultados
print(classification_report(y_test, predicted))

print('------------------------')

print(np.mean(predicted == y_test))

Visualização dos dados:
                                                text  tag
0  film adapt comic book plenti success whether t...    1
1  starter creat alan moor eddi campbel brought m...    1
2  say moor campbel thoroughli research subject j...    1
3  book graphic novel page long includ nearli con...    1
4                       word dont dismiss film sourc    1
------------------------
              precision    recall  f1-score   support

           0       0.72      0.66      0.69      9561
           1       0.69      0.75      0.72      9855

   micro avg       0.70      0.70      0.70     19416
   macro avg       0.71      0.70      0.70     19416
weighted avg       0.71      0.70      0.70     19416

------------------------
0.7047280593325093


#### O Precision e o Recal estão com valores próximos, o classificador está bom

## 3) Utilizando o dataset disposto no portal, faça:
   ## -> extraia o dataset na pasta do notebook
   ## -> crie uma função que leia o conteúdo de cada uma das pastas, amazene num dataframe com duas colunas (review, tag)
   ## -> utilize a classe de tratamento de texto criada acima para tratar o texto
   ## -> crie um pipeline de classificação de texto (countvectorizer/tfidfvectorizer,divisão em treino/teste,instância de modelo, fit e predict)
   ## -> imprima o relatório de classificação
   ## -> OBS: teste várias opções de stemming/lemmatizer

In [18]:
# resposta 3

#Criando o dataFrame
df = pd.DataFrame(columns=['review','tag'])

#Leitura das pastas
for file in os.listdir("pos"):
    pasta = open("pos/"+ file,'r')
    df = df.append({'review':pasta.read(),'tag':'postive'}, ignore_index=True)
        
for file in os.listdir("neg"):
    pasta = open("neg/"+file,'r')
    df = df.append({'review':pasta.read(),'tag':'negative'}, ignore_index=True)
    
print(df.head())

                                              review      tag
0  films adapted from comic books have had plenty...  postive
1  every now and then a movie comes along from a ...  postive
2  you've got mail works alot better than it dese...  postive
3   " jaws " is a rare film that grabs your atten...  postive
4  moviemaking is a lot like being the general ma...  postive


In [19]:
#Com stemização
df_stem = df
df_stem['review'] = [tratarDataFrame(txt, True, False) for txt in df['review']]

print('Visualização')
print(df_stem.head())
print(' ')
vec = TfidfVectorizer()
bow = vec.fit_transform(df_stem['review'])
X_train, X_test, y_train, y_test = train_test_split(bow, df_stem['tag'], test_size=0.3, random_state=69, shuffle = True)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)

print('---stemização---')
print(classification_report(y_test, predicted))
print('Acurácia:',np.mean(predicted == y_test))

Visualização
                                              review      tag
0  film adapt comic book plenti success whether t...  postive
1  everi movi come along suspect studio everi ind...  postive
2  youv got mail work alot better deserv order ma...  postive
3  jaw rare film grab attent show singl imag scre...  postive
4  moviemak lot like gener manag nfl team postsal...  postive
 
---stemização---
              precision    recall  f1-score   support

    negative       0.70      0.85      0.77       287
     postive       0.83      0.67      0.74       313

   micro avg       0.76      0.76      0.76       600
   macro avg       0.77      0.76      0.76       600
weighted avg       0.77      0.76      0.76       600

Acurácia: 0.7566666666666667


In [22]:
#Com lemmatização
df_lem = df
df_lem['review'] = [tratarDataFrame(txt, False, True) for txt in df['review']]

vec = TfidfVectorizer()
bow = vec.fit_transform(df_lem['review'])
X_train, X_test, y_train, y_test = train_test_split(bow, df_lem['tag'], test_size=0.3, random_state=69, shuffle = True)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)
print('---lemmatização---')
print(classification_report(y_test, predicted))
print('Acurácia:',np.mean(predicted == y_test))

---lemmatização---
              precision    recall  f1-score   support

    negative       0.71      0.85      0.78       287
     postive       0.84      0.68      0.75       313

   micro avg       0.76      0.76      0.76       600
   macro avg       0.77      0.77      0.76       600
weighted avg       0.78      0.76      0.76       600

Acurácia: 0.7633333333333333


In [21]:
#Com Ambos
df_lem_stem = df
df_lem_stem['review'] = [tratarDataFrame(txt, True, True) for txt in df['review']]

print('Visualização')
print(df_lem_stem.head())
print(' ')

vec = TfidfVectorizer()
bow = vec.fit_transform(df_lem_stem['review'])
X_train, X_test, y_train, y_test = train_test_split(bow, df_lem_stem['tag'], test_size=0.3, random_state=69, shuffle = True)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)
print('---ambo---')
print(classification_report(y_test, predicted))
print('Acurácia:',np.mean(predicted == y_test))

Visualização
                                              review      tag
0  film adapt comic book plenti success whether t...  postive
1  everi movi come along suspect studio everi ind...  postive
2  youv get mail work alot better deserv order ma...  postive
3  jaw rare film grab attent show singl imag scre...  postive
4  moviemak lot like gener manag nfl team postsal...  postive
 
---ambo---
              precision    recall  f1-score   support

    negative       0.71      0.85      0.78       287
     postive       0.84      0.68      0.75       313

   micro avg       0.76      0.76      0.76       600
   macro avg       0.77      0.77      0.76       600
weighted avg       0.78      0.76      0.76       600

Acurácia: 0.7633333333333333
